## Context Managers and the *with* Statement

built-in функция open() предоставляет нам отличный пример 

In [1]:
with open('hello.txt', 'w') as file:
    file.write('hello, world!')

Код выше транслируется во что-то подобное:
```python
f = open('hello.txt', 'w')
try:
    f.write('hello, world')
finally:
    f.close()
```

Оператор Finally является ключевым, написать вот так - неправильно:
```python
f = open('hello.txt', 'w')
f.write('hello, world')
f.close()
```


Эта реализация не гарантирует, что файл будет закрыт, если во время вызова f.write() возникнет какое-то исключение, что может привести к повреждению дескриптора файла. Именно поэтому оператор with так полезен. Он
упрощает получение и высвобождение ресурсов.

Еще один хороший пример использования with - это threading.Lock class в стандартной библиотеке:

```python
some_lock = threading.Lock()

# некрасиво:
some_lock.acquire()
try:
    # Do something...
finally:
    some_lock.release()
```
    
```python
# лучше:
with some_lock:
    # Do something...
```
В обоих случаях оператор with позволяет абстрагировать внутреннюю логику, вместо написания лишних try и finally.
Контекст менеджер обычно используется для сохранения и восстановления глобальных состояний, блокирования и разблокирования ресурсов, открытия и закрытия файлов.

### Написание своего менеджера

Мы можем реализовать такую же функциональность в собственном
классе. Обычно для этого нужно реализовать методы: \_\_enter\_\_ и \_\_exit\_\_, если мы хотим, чтобы он функционировал как менджер контекста. Python сам вызовет эти функции в нужный момент. 

In [3]:
class ManagedFile:
    
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close() 

In [8]:
with ManagedFile('hello.txt') as f:
    f.write('hello, world!\n')
    f.write('bye now')

In [9]:
with open('hello.txt', "r") as f:
    print(f.readlines())

['hello, world!\n', 'bye now']


## Underscores, Dunders, and More

1. Single Leading Underscore: _var
2. Single Trailing Underscore: var_
3. Double Leading Underscore: \_\_var
4. Double Leading and Trailing Underscore: \_\_var\_\_
5. Single Underscore: _

### 1. Single Leading Underscore: _var
Только внутреннее использование!

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        
t = Test()
print(t.foo)
print(t._bar) # ?

```python
def external_func():
    return 23

def _internal_func():
    return 42
```
---
```python
>>> from my_module import *
>>> external_func()
23
>>> _internal_func()
NameError: "name '_internal_func' is not defined"
```
---
```python
>>> import my_module
>>> my_module.external_func()
23
>>> my_module._internal_func()
42
```

### 2. Single Trailing Underscore: var_
Если имя занято самим python'ом у нас есть способ разрешить этот конфликт

In [ ]:
def make_object(name, class):
    pass

In [ ]:
def make_object(name, class_):
    pass

### 3. Double Leading Underscore: \_\_var
Двойной underscore заставляет интерпретатор Python перезаписывать
имя атрибута, чтобы избежать конфликтов имен в подклассах.

In [ ]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 23
        
t = Test()
dir(t)

Где self.\_\_baz???

In [ ]:
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'

t2 = ExtendedTest()
print(t2.foo)
print(t2._bar)
print(t2.__baz)

Почему так происходит? 

Процесс называется name mangling (искажение имени?), и используется интерпретатором Python, чтобы защитить переменную от переопределения в подклассах.

In [ ]:
dir(t2)

In [ ]:
print(t2._ExtendedTest__baz)
print(t2._Test__baz)

*Двойное подчеркивание имен полностью прозрачно для программиста:*

In [ ]:
class ManglingTest:
    
    def __init__(self):
        self.__mangled = 'hello'
        
    def get_mangled(self):
        return self.__mangled
    
print(ManglingTest().get_mangled())
print(ManglingTest().__mangled)

### 4. Double Leading and Trailing Underscore: __var__
Может показаться удивительным, но name mangling не применяется, если имя начинается и заканчивается двойным подчеркиванием:

In [ ]:
class PrefixPostfixTest:
    def __init__(self):
        self.__bam__ = 42
        
print(PrefixPostfixTest().__bam__)

Эти имена зарезервированы в Python для специального использования (например: \_\_init\_\_, \_\_call\_\_.
Лучшей практикой является не использовать такие имена в своих программах

### 5. Single Underscore: _
В соответствии с соглашением, одиночное подчеркивание иногда используется как
имя, указывающее, что переменная является временной или несущественной.

### Задание 1 Привидите пример кода с Single Underscore: _


### Задание 2 Реализуйте менеджер контекста для измерения времени работы кода
Пример использования:
``` python
import httplib

with Timer() as t:
    conn = httplib.HTTPConnection('google.com')
    conn.request('GET', '/')

print('Request took %.03f sec.' % t.interval)
```

### Задание 3 Придумайте и реализуйте свой вариант менеджера контекста with:


### Бонус: для создания менеджера контекста можно использовать декоратор из библиотеки contextmanager
```python
from contextlib import contextmanager


@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

>>> with managed_file('hello.txt') as f:
...     f.write('hello, world!')
...     f.write('bye now')
```